In [ ]:
%pip install google-auth-oauthlib
%pip install --upgrade google-api-python-client

In [4]:
%pip install aiofiles

Note: you may need to restart the kernel to use updated packages.


In [1]:
import aiohttp
import aiofiles
import json
from google.oauth2.service_account import Credentials
from datetime import datetime, timedelta

async def get_views(page_path: str):
    SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
    
    async with aiofiles.open('/home/anon-b/Downloads/toknowviews-1730747751593-d5a762fdba11.json', mode='r') as f:
        service_account_info = json.loads(await f.read())
    
    credentials = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
    token = credentials.token
    
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    body = {
        "dateRanges": [{
            "startDate": "2024-01-01",
            "endDate": tomorrow_date
        }],
        "dimensions": [{
            "name": "pagePath"
        }],
        "dimensionFilter": {
            "filter": {
                "fieldName": "pagePath",
                "stringFilter": {
                    "matchType": "CONTAINS",
                    "value": page_path
                }
            }
        },
        "metrics": [{
            "name": "screenPageViews"
        }]
    }
    
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    async with aiohttp.ClientSession() as session:
        async with session.post('https://analyticsdata.googleapis.com/v1beta/properties/447265843:runReport', headers=headers, json=body) as response:
            response_json = await response.json()
    
    counts = [
        int(metricValue.get('value', 0))
        for row 
        in response_json.get('rows', []) 
            for metricKey, metricValues 
            in row.items() 
                for metricValue 
                in metricValues
                if metricKey == 'metricValues'
    ]
    return sum(counts)

In [5]:
await get_views("/about")

0

<https://gemini.google.com/app/9a1a99a1df31aab5>

<https://claude.ai/chat/4309b5b3-dae7-44d1-ab94-4e3cd4a21633>

<https://claude.ai/chat/30125116-bf87-42b5-9d71-09962e619a7e>

<https://developers.cloudflare.com/workers/languages/python/packages/#supported-packages>